# The Roboat environmental data API Wrapper

The following is an very, very small intro into the Roboat environmental API wrapper. It should give you a good idea of what is possible and will be completely documented in due time.

In the near future, the wrapper will be updated with object functionality to make everything super fluid - in the mean time, we can get by with this simple API wrapper with somewhat limited functionality and little documentation...


## Authentication

To get started, you must first request an API Key. You can do so at https://www.roboat-enviro.com/api-keys. Once you have your API key, store it as an environment variable called *ROBOAT-ENVIRO_APIKEY*.

In [ ]:
import roboatenviro
print ("roboatenviro v{}".format(roboatenviro.__version__))

## Advanced Queries

The API itself has quite powerful querying capabilites, though they can be a bit confusing to use at first. Eventually, I will get around to writing a "helper" function to make this as simple as possible.

### Limit

To limit the number of results you get for queries where a list of information is returned, you can add the `limit` keyword to your `params` dictionary. The argument must be an integer (i.e. `limit=5`.)

Example:

Return the first five sensors in the list:

    >>> api.get_sensors(params=dict(limit=5))
    
### Sort

You can sort the order of results by any column and either by `asc` or `desc`. The format for the `sort` keyword must be `sort="[column],[asc or desc]"`. You can also join multiple sorts together using a semicolon.

Example:

Return the first five sensors, sorted asc by the last seen column:

    >>> api.get_sensors(params=dict(limit=5, sort="last_seen,asc"))
    
  
Return the first five sensors, sorted asc by last_seen and asc by username:

    >>> api.get_sensors(params=dict(limit=5, sort="last_seen,asc;username,asc"))
    

### Filter

Filter is probably the most useful because it allows you to grab data between certain timestamps! There is also a ton of embedded functionality built in. The arguments that can be used include:

  * `eq`: equals
  * `ne`: not equals
  * `lt`: less than
  * `le`: less than or equal to
  * `gt`: greater than
  * `ge`: greater than or equal to
  * `in`: in
  * `like`: like
  
The format to the argument is as follows: `filter="[column],[arg],[value]"`. 

Examples:

Grab data between May 1st and June 1st, 2018.

    >>> api.get_data(sn="SN001", params=dict(filter="timestamp,ge,2018-05-01;timestamp,lt,2018-06-01"))


## Initialize a RoboatEnviro object

To start, we will first setup an object of the manager, called `roboatenviro.RoboatEnviro`. If you haven't stored your api key as an environment variable, or have called it something else, feel free to enter it here.

In [ ]:
import os
# get the token from a different environment variable
#token = os.getenv("ROBOAT-ENVIRO_APIKEY_DEV")

#hardcoded in here for now...
token = "token"

# set up the roboatenviro.RoboatEnviro object
# here, we are also going to manually set the endpoint because I am doing this tutorial on 
# localhost...
api = roboatenviro.RoboatEnviro(token=token, endpoint="http://localhost:5000/api/")

## `roboatenviro.legacy.RoboatEnviro(*args, **kwargs)`

The RoboatEnviro (can also be called the 'manager') object provides a simple way to view information about all the sensors you have access to with your account. Remember, the API key is tied to your account, so don't share the key with anyone!  Please keep it safe and sound.

Below, I will detail a few of the methods available from the manager:

### `roboatenviro.legacy.RoboatEnviro.get_account(return_type='json')`

The `get_account` method simply retrieves the account information tied to your API key as a dictionary.

In [ ]:
api.get_accounts(return_type='dataframe')

### `roboatenviro.legacy.RoboatEnviro.get_sensors(return_type='json', **kwargs)`

With the `get_sensors` method, we can retrieve all the sensors we have access to. You can return them either as 'json' or 'dataframe'.

#### json

In [ ]:
api.get_sensors()

#### dataframe

In [ ]:
api.get_sensors(return_type='dataframe')

You should also be able to send whatever query params you want as keywords - see above in the "Advanced Queries" section for more information.

In [ ]:
api.get_sensors(return_type='dataframe', params=dict(limit=1))

### `roboatenviro.legacy.RoboatEnviro.get_sensor(sn)`

You can get a single sensor, by using this method.

In [ ]:
api.get_sensor(sn="SN001")

### `roboatenviro.legacy.RoboatEnviro.update_sensor(sn, params)`

Use this method to update a sensor with params as a dictionary.

In [ ]:
#params = dict(lat=12.1, lon=-100)
params = dict(sn="SN002")
api.update_sensor("SN001", params=params)
params = dict(sn="SN001")
api.update_sensor("SN002", params=params)

## `roboatenviro.legacy.RoboatEnviro.get_data(sn, return_type="json", final_data=True, **kwargs)`

Use this method to retrieve a list of data for a given SN.

In [ ]:
import json
from pprint import pprint
import pandas as pd
from datetime import datetime

df = pd.read_csv("trf_upload.csv")
df["scan_datetime"] = datetime.utcnow().isoformat()
df.drop(["sensor_sn"], axis=1, inplace=True)
params = {
    "sensor_sn": "SN001",
    "start_wl": int(df["mcr_wl"].min()),
    "end_wl": int(df["mcr_wl"].max()),
    "start_datetime": df["scan_datetime"].min(),
    "end_datetime": df["scan_datetime"].max()
}
scan_set = api.add_trf_scan_set(params=params)
scan_set = json.loads(scan_set.text)
scan_set_id = scan_set['id']
print("scan_set_id: ", scan_set_id)
display(api.get_trf_scan_set("SN001", return_type="dataframe"))

df['scan_set_id'] = scan_set_id
display(df.head())
for idx, row in df.iterrows():
    params = row.to_dict()
    print(api.add_trf_data(params=params))

'''
trf_scan_set = {
    "sensor_sn": "SN001",
    "start_wl": df["mcr_wl"].min(),
    "end_wl": df["mcr_wl"].max(),
    "trf_scans": df.to_dict('records')
}
pprint(trf_scan_set)
'''

In [ ]:
api.get_trf_data("SN001", return_type="dataframe")

In [ ]:
# get the 'research' data - must have sufficient permissions
api.get_trf_data(sn="SN001", return_type="dataframe", params=dict(limit=10))

Get the last 5 datapoints since March 15th:

In [ ]:
api.get_trf_data(sn="SN001", return_type="dataframe",
             params=dict(filter="scan_datetime,gt,2020-01-08 17:46:19"))

In [ ]:
api.get_ssf_data(sn="SN001", return_type='json')

## `roboatenviro.legacy.RoboatEnviro.get_logs(sn, return_type="json", **kwargs)`

Use this method to retrieve a list of logs for a given SN.

In [ ]:
#CRUD sensor logs
api.get_sensor_logs(return_type='dataframe')

In [ ]:
#CRUD deployments
api.get_deployments(return_type='dataframe')